In [1]:
import os
import glob
import time
import datetime
import collections

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as snc

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

import PIL
import lightgbm

In [2]:
%matplotlib inline

In [3]:
scores_df = pd.read_csv('../input/depression/data/scores.csv')

In [4]:
#scores_df

In [5]:
#scores_df_cond_only

In [6]:
#scores_df_cond_only = scores_df.drop([23:])
scores_df = scores_df.drop(['melanch', 'edu'], axis=1)
#scores_df = scores_df.drop(['edu'], axis=1)

In [7]:
#scores_df

In [8]:
#scores_df_cond_only = pd.DataFrame()

In [9]:
#scores_df_cond_only = scores_df if 'condition' in scores_df['number'] else 0

In [10]:
#for i, entry in enumerate(scores_df['number']):
#    if 'condition' in entry:
#        scores_df_cond_only = pd.concat([scores_df_cond_only, scores_df.iloc[i]], axis=0)

In [11]:
#scores_df_cond_only

In [12]:
#train_X_timestamp.iloc[i, 'timestamp'] = np.dtype(splitted_timestamp[0], np.datetime_data)

In [13]:
def get_data(path_to_data, scores_df):
    data_df = pd.read_csv('../input/depression/data/condition/condition_1.csv')

    for filename in scores_df['number'][1:]:
        #print(filename)
        if 'condition' in filename:
            #print(path_to_data + filename + '.csv')
            #data_df = pd.concat([pd.read_csv(path_to_data + filename + '.csv'), data_df], axis=0)
            #data_df = pd.merge([data_df, scores_df], 'right')
            data_df = pd.concat([data_df, pd.read_csv(path_to_data + filename + '.csv')], axis=0)
    data_df.reset_index(inplace=True)
    return data_df[data_df.columns.drop(['index'])]

In [19]:
#def get_data(path_to_data):
#   return pd.read_csv(path_to_data)

In [ ]:
#def get_labels(path_to_labels):
#   return pd.read_csv(path_to_labels)

In [20]:
train_df = get_data('../input/depression/data/condition/', scores_df)

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551716 entries, 0 to 551715
Data columns (total 3 columns):
timestamp    551716 non-null object
date         551716 non-null object
activity     551716 non-null int64
dtypes: int64(1), object(2)
memory usage: 12.6+ MB


In [16]:
#train_df.head()

In [17]:
#train_df.tail()
#train_df.reset_index(inplace=True)

In [18]:
train_df.shape

(551716, 3)

In [21]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551716 entries, 0 to 551715
Data columns (total 3 columns):
timestamp    551716 non-null object
date         551716 non-null object
activity     551716 non-null int64
dtypes: int64(1), object(2)
memory usage: 12.6+ MB


In [22]:
le = LabelEncoder()

In [35]:
clf_lr = LogisticRegression(solver='lbfgs' ,multi_class='multinomial', n_jobs=12)
#clf = DecisionTreeClassifier(min_impurity_decrease=0.001)
clf_lgb = lightgbm.LGBMClassifier(objective='multiclass', max_depth=4, learning_rate=0.01, num_threads=12)

In [24]:
#train_X = pd.to_datetime(train_df[train_df.columns.drop(['timestamp', 'activity',])])
#train_X = (pd.to_datetime(train_df[train_df.columns.drop(['timestamp', 'activity'])])).dt.strftime('%Y-%m-%d  %H:%M%:%S').astype(np.float32)
#train_X = train_df[train_df.columns.drop(['timestamp', 'activity'])].split('-').astype(np.float32)
train_X = le.fit_transform(train_df[train_df.columns.drop(['date', 'activity'])])
#print(le.fit_transform(train_df[train_df.columns.drop(['date', 'activity'])]))
#train_X = 
#train_X_timestamp = pd.to_datetime(train_X.loc['date', :])
#train_X_timestamp.loc['timestamp', :] = pd.to_datetime(train_X.loc['timestamp', :])
#train_X_timestamp['timestamp'] =
#pd.to_datetime(train_X.loc['timestamp', :])
#temp_train_X = pd.DataFrame(index=[0:551715])
#temp_train_X = pd.DataFrame()
#max_idx = 0
'''
for i, date in enumerate(train_X['date']):
    #if i > max_idx:
    #    max_idx = i
    #splitted_timestamp = timestamp.split(' ')
    #train_X_timestamp.iloc[i, 'timestamp'] = np.dtype(splitted_timestamp[0], np.datetime_data)
    train_date = pd.to_datetime(date)
    train_X_timestamp.loc['date', :] = train_date
    
    #train_X_timestamp.iloc[i, 'timestamp'] = timestamp
'''
#print(max_idx)
train_y = train_df[train_df.columns.drop(['timestamp', 'date'])]

In [31]:
train_X[:10]

array([93298, 93299, 93300, 93301, 93302, 93303, 93304, 93305, 93306,
       93307])

In [32]:
train_X[-10:]

array([50285, 50286, 50287, 50288, 50289, 50290, 50291, 50292, 50293,
       50294])

In [33]:
train_X_reshaped = train_X.reshape(-1, 1)

In [34]:
train_X_reshaped.shape

(551716, 1)

In [29]:
start_time = time.time()
clf_lgb.fit(train_X_reshaped, train_y)
print("total_time in min: {}".format((time.time() - start_time ) / 60))

total_time in min: 13.780151077111562


In [36]:
start_time = time.time()
clf_lr.fit(train_X_reshaped, train_y)
print("total time in min: {}".format((time.time() - start_time ) / 60)) 

total time in min: 75.6103862563769
